In [1]:
import numpy as np
import matplotlib as plt
from hilp import load_csv_data
from helper import *
from costs import *
from least_squares import *
from ridge_regression import *
from logistic_regression import *

In [2]:
yb, input_data, ids = load_csv_data('data/test.csv')

In [3]:
input_data.shape

(568238, 30)

In [4]:
input_data

array([[-999.   ,   79.589,   23.916, ..., -999.   , -999.   ,    0.   ],
       [ 106.398,   67.49 ,   87.949, ..., -999.   , -999.   ,   47.575],
       [ 117.794,   56.226,   96.358, ..., -999.   , -999.   ,    0.   ],
       ...,
       [ 108.497,    9.837,   65.149, ..., -999.   , -999.   ,    0.   ],
       [  96.711,   20.006,   66.942, ..., -999.   , -999.   ,   30.863],
       [  92.373,   80.109,   77.619, ..., -999.   , -999.   ,    0.   ]])

In [5]:
initial_w = np.zeros(30)

In [18]:
cleaned_columns = [c for c in input_data.T if (c==-999).sum()/len(c) < 0.2]
print(np.shape(cleaned_columns))
for c in cleaned_columns:
    numb_of_nan = (c==-999).sum()
    median = np.median(list(filter(lambda x : x!= -999, c)))
    c[c == -999] = median

(20, 568238)
86488
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [19]:
cleaned_columns

[array([112.541, 106.398, 117.794, ..., 108.497,  96.711,  92.373]),
 array([79.589, 67.49 , 56.226, ...,  9.837, 20.006, 80.109]),
 array([23.916, 87.949, 96.358, ..., 65.149, 66.942, 77.619]),
 array([ 3.036, 49.994,  4.137, ..., 18.006, 29.761,  3.984]),
 array([0.903, 2.048, 2.755, ..., 2.742, 2.479, 2.486]),
 array([ 3.036,  2.679,  4.137, ..., 18.006,  2.739,  3.984]),
 array([ 56.018, 132.865,  97.6  , ...,  68.097, 101.676,  77.348]),
 array([1.536, 1.777, 1.096, ..., 0.642, 0.936, 1.683]),
 array([-1.404, -1.204, -1.408, ..., -0.836,  1.41 , -1.346]),
 array([22.088, 30.716, 46.564, ..., 41.478, 36.575, 28.83 ]),
 array([-0.54 , -1.784, -0.298, ...,  1.057, -1.249,  1.8  ]),
 array([-0.609,  3.054,  3.079, ...,  1.025, -1.54 , -1.138]),
 array([33.93 , 54.574, 51.036, ..., 26.619, 34.238, 48.518]),
 array([-0.504, -0.169, -0.548, ...,  1.087, -1.216,  0.846]),
 array([-1.511,  1.795,  0.336, ..., -1.717,  0.939,  2.85 ]),
 array([48.509, 21.093, 19.461, ..., 30.463,  6.647, 35

In [24]:
initial_w = np.random.randint(0,1,21)
batch_size = 1
max_iters = 20
#gamma = 0.01
lambda_ = 0.01

In [25]:
gammas = [0.001, 0.002, 0.005, 0.01, 0.02, 0.03]

In [30]:
# standardize the data
std_data, mean, std = standardize(np.transpose(cleaned_columns))

In [31]:
# build the model (add 1's for the w0)
tx, y = build_model_data(std_data, yb)

In [32]:
l = []
W = []
for gamma in gammas:
    loss, ws = gradient_descent(y, tx, initial_w, max_iters, gamma)
    l.append(loss[max_iters-1])
    W.append(ws)
best_loss = min(l)
ind = l.index(best_loss)
print("best loss :" + str(best_loss)+ " " + str(gammas[ind]) +" \n "+ str(W[ind]))

GD iter. 0/19: loss=0.5, w0=0.001, w1=0.0009808290245267213
GD iter. 1/19: loss=0.48781280561833645, w0=0.0019877176623243637, w1=0.0019485697448016828
GD iter. 2/19: loss=0.4760000389919894, w0=0.0029633432735610373, w1=0.0029034249717402702
GD iter. 3/19: loss=0.4645496590569508, w0=0.0039270640349752285, w1=0.003845594229914548
GD iter. 4/19: loss=0.45345001528982914, w0=0.004879064112959207, w1=0.00477527381128266
GD iter. 5/19: loss=0.4426898350211213, w0=0.005819524688655897, w1=0.005692656828038347
GD iter. 6/19: loss=0.4322582111607329, w0=0.006748624006769798, w1=0.006597933264594977
GD iter. 7/19: loss=0.4221445903223509, w0=0.007666537423578546, w1=0.007491290028718276
GD iter. 8/19: loss=0.41233876133370473, w0=0.008573437454158204, w1=0.008372911001821701
GD iter. 9/19: loss=0.4028308441201783, w0=0.009469493818835183, w1=0.009242977088438175
GD iter. 10/19: loss=0.3936112789496409, w0=0.010354873488877425, w1=0.010101666264881674
GD iter. 11/19: loss=0.38467081602675485, 

GD iter. 18/19: loss=0.026344717969364415, w0=0.11587276552885302, w1=0.09183992527627102
GD iter. 19/19: loss=0.024451099459917042, w0=0.11808803160932539, w1=0.09260734482979892
GD iter. 0/19: loss=0.5, w0=0.03, w1=0.029424870735801637
GD iter. 1/19: loss=0.21715883430655794, w0=0.048945896091927385, w1=0.047070267645018316
GD iter. 2/19: loss=0.1306582678464032, w0=0.06197542615032086, w1=0.058412085443093606
GD iter. 3/19: loss=0.09717230125554797, w0=0.07172721663217073, w1=0.06626428075366154
GD iter. 4/19: loss=0.07925485508198506, w0=0.07956671270295491, w1=0.07208449666439815
GD iter. 5/19: loss=0.06690564651232433, w0=0.08620980364123201, w1=0.07663824496149957
GD iter. 6/19: loss=0.05727600660033112, w0=0.0920401417165649, w1=0.08033687662421823
GD iter. 7/19: loss=0.04941155616717968, w0=0.09727067915297716, w1=0.08340964561370863
GD iter. 8/19: loss=0.04288498727831133, w0=0.10202597045789627, w1=0.08599138706125728
GD iter. 9/19: loss=0.03743613682825105, w0=0.10638416782

In [33]:
stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma)

SGD iter. 0/19: loss=0.5, w0=0.03, w1=0.014756625182948614
SGD iter. 1/19: loss=0.302184581738194, w0=0.05475032632206361, w1=0.033149890718620456
SGD iter. 2/19: loss=0.17633403988669244, w0=0.0725442767284131, w1=0.05249719629733035
SGD iter. 3/19: loss=0.09154653019588929, w0=0.08495045530141379, w1=0.0629140279407279
SGD iter. 4/19: loss=0.058867832543728904, w0=0.09328450738974683, w1=0.0660188410569813
SGD iter. 5/19: loss=0.04788673849566266, w0=0.09633931939512004, w1=0.06786747224983986
SGD iter. 6/19: loss=0.04867127737946477, w0=0.10374513152716125, w1=0.07481194791319942
SGD iter. 7/19: loss=0.05021146359630607, w0=0.10311104468532918, w1=0.074255275762211
SGD iter. 8/19: loss=0.04962718719841374, w0=0.11156514160987993, w1=0.07924195781684461
SGD iter. 9/19: loss=0.05120830306934027, w0=0.1213281978532822, w1=0.08403323257027688
SGD iter. 10/19: loss=0.05409415619242854, w0=0.10423340353464525, w1=0.06458251560838654
SGD iter. 11/19: loss=0.06942078905056302, w0=0.12162710

([0.5,
  0.302184581738194,
  0.17633403988669244,
  0.09154653019588929,
  0.058867832543728904,
  0.04788673849566266,
  0.04867127737946477,
  0.05021146359630607,
  0.04962718719841374,
  0.05120830306934027,
  0.05409415619242854,
  0.06942078905056302,
  0.02875469599741479,
  0.02801105597323452,
  0.029195048241815093,
  0.03289549247831841,
  0.0357219259196171,
  0.02043652946657889,
  0.01761733958087498,
  0.01710805641448588],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([ 0.03      ,  0.01475663,  0.00570703,  0.00993637, -0.00608797,
         -0.0166621 , -0.00608797,  0.01295732, -0.01734942, -0.01813471,
          0.0013766 , -0.01675343, -0.01661058, -0.00600718, -0.01680651,
         -0.01813862, -0.0034019 , -0.01724872,  0.03316689, -0.0175879 ,
         -0.0175879 ]),
  array([ 0.05475033,  0.03314989,  0.00718181,  0.01612144, -0.00885625,
         -0.03034153, -0.02015059,  0.03159402, -0.03152152, -0.03223243,
         -0.00

In [34]:
least_squares(y, tx)

(array([ 9.98405457e-01, -2.63516752e-05, -1.50330784e-05,  1.09484863e-05,
        -4.39850310e-05, -1.52511218e-03,  2.33867695e-06,  5.45501709e-04,
        -2.50318251e-04, -4.14133501e-06, -5.37872314e-04, -9.30016176e-06,
        -4.29734428e-06, -5.22613525e-04,  3.31056071e-10, -2.38287612e-10,
         2.41016096e-11, -2.47837306e-11, -3.12638804e-12,  2.19733920e-09,
        -5.30242920e-04]),
 5.669917595398336e-10)

In [35]:
ridge_regression(y, tx, lambda_)

array([ 0.22013326,  0.0128634 , -0.00115374, -0.00903955, -0.00794017,
       -0.12259335, -0.02732789,  0.0641795 , -0.12533721, -0.12890241,
       -0.07021076, -0.12660068, -0.12627105, -0.0552304 , -0.12648365,
       -0.12595749, -0.00439477, -0.12537163,  0.01264825, -0.12686137,
       -0.06849141])

In [36]:
logistic_regression(y, tx, initial_w, max_iters, gamma)

Logistic regression iter. 0/19: loss=0.5299255963296047, w0=0.03, w1=0.029424870735801637
Logistic regression iter. 1/19: loss=0.42945258897202265, w0=0.054593135785639985, w1=0.05310888019509369
Logistic regression iter. 2/19: loss=0.3620579431388673, w0=0.0753633148935794, w1=0.07279384743977052
Logistic regression iter. 3/19: loss=0.31358822619978055, w0=0.09336537572140455, w1=0.08963149014153658
Logistic regression iter. 4/19: loss=0.27693804361581015, w0=0.10927570080849464, w1=0.10434969565842568
Logistic regression iter. 5/19: loss=0.24818506046416414, w0=0.12354733322279314, w1=0.11742904877243247
Logistic regression iter. 6/19: loss=0.22498516227849194, w0=0.13649810935725748, w1=0.1292023243742625
Logistic regression iter. 7/19: loss=0.20584758303474265, w0=0.14835988374243608, w1=0.1399094454497421
Logistic regression iter. 8/19: loss=0.18977662632639716, w0=0.15930726592219002, w1=0.1497292115398588
Logistic regression iter. 9/19: loss=0.17608054206339668, w0=0.16947523744

([0.5299255963296047,
  0.42945258897202265,
  0.3620579431388673,
  0.31358822619978055,
  0.27693804361581015,
  0.24818506046416414,
  0.22498516227849194,
  0.20584758303474265,
  0.18977662632639716,
  0.17608054206339668,
  0.16426298361573355,
  0.15395810029025878,
  0.14489001769438986,
  0.13684660730063603,
  0.1296619709014058,
  0.12320442655213656,
  0.11736807286440219,
  0.1120667425134278,
  0.10722958829118928,
  0.10279780784305365],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([ 0.03      ,  0.02942487,  0.00163787,  0.0140745 ,  0.00498305,
         -0.01666123, -0.01017499,  0.04434103, -0.01702614, -0.0176374 ,
         -0.00248543, -0.01759257, -0.01759389,  0.00064189, -0.01759527,
         -0.01756758, -0.00134072, -0.01759101,  0.06435955, -0.0172053 ,
          0.01100878]),
  array([ 0.05459314,  0.05310888,  0.00315119,  0.02535277,  0.00733476,
         -0.03031034, -0.01882713,  0.07694681, -0.03098609, -0.03211366,
 

In [17]:
reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma)

Reg Logistic regression iter. 0/19: loss=0.482883941928351, w0=0.03, w1=0.008734977955419705
Reg Logistic regression iter. 1/19: loss=0.3813995161179354, w0=0.05273010186764367, w1=0.015969413408945303
Reg Logistic regression iter. 2/19: loss=0.3226150435299309, w0=0.07123888045221399, w1=0.022253667672864588
Reg Logistic regression iter. 3/19: loss=0.2833188444211394, w0=0.08712330137763158, w1=0.027889769444266743
Reg Logistic regression iter. 4/19: loss=0.25444588053529144, w0=0.10120683442132598, w1=0.03304100030976658
Reg Logistic regression iter. 5/19: loss=0.23187191791977668, w0=0.11395531376526412, w1=0.03780455839834982
Reg Logistic regression iter. 6/19: loss=0.21346723742430662, w0=0.12565710049123108, w1=0.042244066043404825
Reg Logistic regression iter. 7/19: loss=0.19801665687541226, w0=0.13650442441207564, w1=0.04640454055597918
Reg Logistic regression iter. 8/19: loss=0.18476958365589788, w0=0.1466332798478942, w1=0.050319834853336534
Reg Logistic regression iter. 9/19

([0.482883941928351,
  0.3813995161179354,
  0.3226150435299309,
  0.2833188444211394,
  0.25444588053529144,
  0.23187191791977668,
  0.21346723742430662,
  0.19801665687541226,
  0.18476958365589788,
  0.17323179990733711,
  0.1630605349076253,
  0.15400773683235203,
  0.14588753493001916,
  0.1385566069633561,
  0.13190180010939564,
  0.1258320176674251,
  0.12027271671616714,
  0.11516206174290736,
  0.11044816269801415,
  0.10608704401375968],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([ 0.03      ,  0.00873498,  0.01551733,  0.01772283,  0.01611056,
         -0.03685872, -0.02940337, -0.03692395,  0.01227217,  0.01342244,
          0.02309028,  0.01220746,  0.01209906, -0.03689794,  0.0147861 ,
          0.01210701,  0.01210678,  0.0153407 ,  0.01210653,  0.01211144,
          0.0149891 ,  0.01210729,  0.02664036,  0.01217569, -0.01204496,
         -0.01557085, -0.01557174, -0.03574102, -0.03690727, -0.